In [1]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 511, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 511 (delta 159), reused 124 (delta 91), pack-reused 269 (from 1)
Receiving objects: 100% (511/511), 163.95 KiB | 5.65 MiB/s, done.
Resolving deltas: 100% (261/261), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.6 MB/s eta 0:00:00
Installing RAPIDS remaining 24.4.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.3/473.3 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.9/920.9 kB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Imputation

/content/drive/MyDrive/Imputation


In [4]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


In [5]:
import numpy as np
import torch
import optuna
from sklearn.impute import SimpleImputer
from cuml.svm import SVR, LinearSVR
from utils import masked_mae_cal
from naive_models import read_files

In [6]:
DATASET = 'USGSOhioRiver'           #choose from USGSMuddyFK, USGSOhioRiver
NUM_ITERATIONS = 10
CONVERGENCE_THRESHOLD = 1e-4
MISSING_PERCENTAGE = 20
KERNEL = 'rbf'       #choose from linear and rbf

In [7]:
def introduce_missingness_train():
    #Load split and scaled train data
    X = read_files(DATASET,'train',MISSING_PERCENTAGE)
    l,m,n = X.shape[0],X.shape[1], X.shape[2]
    X = X.reshape(-1)
    indices = np.where(~np.isnan(X))[0].tolist()
    indices = np.random.choice(
        indices,
        int(len(indices) * MISSING_PERCENTAGE/100),
        replace=False
    ).astype(int)
    X_hat = np.copy(X)
    X_hat[indices] = np.nan
    missing_mask = (~np.isnan(X_hat)).astype(np.float32)    #Gives 1 if observed and 0 otherwise
    indicating_mask = ((~np.isnan(X_hat)) ^ (~np.isnan(X))).astype(np.float32) #Gives 1 if artificially missing 0 otherwise
    X, X_hat, missing_mask, indicating_mask = [x.reshape(l*m, n) for x in
                                               (X, X_hat, missing_mask, indicating_mask)]
    return X, X_hat, missing_mask, indicating_mask

In [8]:
class MICEImputation:
    def __init__(self, X, missing_mask):
        self.X = X
        self.missing_mask = missing_mask

    def init_imputation(self):
        imputer = SimpleImputer(strategy='mean')
        initial_imputations = imputer.fit_transform(self.X)
        return initial_imputations

    def regression_imputation(self,imputations,target_col_idx,missing_mask,model):
        #X_train, y_train and X_test are nomenclatures
        mice_mask = (1-missing_mask).astype(bool)
        target_missing_mask = mice_mask[:,target_col_idx]
        X_train = imputations[~target_missing_mask,:]
        y_train = imputations[~target_missing_mask,target_col_idx]
        X_train = np.delete(X_train,target_col_idx,axis=1)
        X_val = imputations[target_missing_mask,:]
        X_val = np.delete(X_val,target_col_idx,axis=1)
        if X_train.shape[0]>0:
            y_train = y_train.ravel()
            model.fit(X_train,y_train)
            predictions = model.predict(X_val)
            imputations[target_missing_mask,target_col_idx] = predictions
        return imputations

In [9]:
def mice_run(trial):
    C = trial.suggest_categorical('C',[0.01, 0.1, 1.])
    epsilon = trial.suggest_categorical('epsilon', [0.01, 0.1])
    if KERNEL == 'linear':
      model = LinearSVR(C = C,
                        epsilon = epsilon)
    else:
      gamma = trial.suggest_categorical('gamma', ['scale', 'auto', 0.001])
      model = SVR(kernel = KERNEL,
                  gamma = gamma,
                  epsilon = epsilon,
                  C = C)
    X, X_hat, missing_mask, indicating_mask = introduce_missingness_train()
    miceimputation = MICEImputation(X,missing_mask)
    imputations = miceimputation.init_imputation()
    for iteration in range(NUM_ITERATIONS):
        prev_imp = imputations.copy()
        for j in range(X.shape[1]):
            imputations = miceimputation.regression_imputation(imputations,j,missing_mask,model)
            change_in_imputations = np.max(np.abs(imputations-prev_imp),axis=0)
            if np.all(change_in_imputations<CONVERGENCE_THRESHOLD):
                break
    return masked_mae_cal(torch.from_numpy(imputations),torch.from_numpy(np.nan_to_num(X)),torch.from_numpy(indicating_mask))

In [ ]:
if __name__=="__main__":
    if KERNEL == 'linear':
      search_space = {
                'C':[0.01, 0.1, 1.],
                'epsilon':[0.01, 0.1]
                }
      study = optuna.create_study(direction='minimize',sampler=optuna.samplers.GridSampler(search_space=search_space),storage=f"sqlite:///MICE_LinearSVR.sqlite3",study_name=f"{DATASET}")
    else:
      search_space = {
                'C':[0.01, 0.1, 1.],
                'epsilon':[0.01, 0.1],
                'gamma':['scale', 'auto', 0.001]
                }
      study = optuna.create_study(direction='minimize',sampler=optuna.samplers.GridSampler(search_space=search_space),storage=f"sqlite:///MICE_RBFSVR.sqlite3",study_name=f"{DATASET}_1")
    study.optimize(mice_run)
    print("Study statistics: ")
    print(" Number of finished trials: ",len(study.trials))
    print("Best trial")
    trial = study.best_trial
    print(" Value: ",trial.value)
    print("Params: ")
    for key,value in trial.params.items():
        print("   {}: {}".format(key,value))

[I 2024-10-06 17:53:26,754] A new study created in RDB with name: USGSOhioRiver_1
[I 2024-10-06 18:04:50,427] Trial 0 finished with value: 0.30413782596588135 and parameters: {'C': 0.01, 'epsilon': 0.01, 'gamma': 'auto'}. Best is trial 0 with value: 0.30413782596588135.
[I 2024-10-06 18:17:17,512] Trial 1 finished with value: 0.26151731610298157 and parameters: {'C': 0.1, 'epsilon': 0.01, 'gamma': 'scale'}. Best is trial 1 with value: 0.26151731610298157.
[I 2024-10-06 18:29:04,210] Trial 2 finished with value: 0.4283367395401001 and parameters: {'C': 0.1, 'epsilon': 0.01, 'gamma': 0.001}. Best is trial 1 with value: 0.26151731610298157.
[I 2024-10-06 18:34:34,133] Trial 3 finished with value: 0.28841090202331543 and parameters: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'auto'}. Best is trial 1 with value: 0.26151731610298157.
[I 2024-10-06 18:45:04,606] Trial 4 finished with value: 0.4159254729747772 and parameters: {'C': 1.0, 'epsilon': 0.01, 'gamma': 0.001}. Best is trial 1 with value: 0.

Study statistics: 
 Number of finished trials:  18
Best trial
 Value:  0.24320194125175476
Params: 
   C: 1.0
   epsilon: 0.01
   gamma: scale
